# FastSMC example notebook

This notebook demonstrates how to use the FastSMC python bindings.

Please make sure you have installed the python bindings by following the instructions in `../README.md` before attempting to run this notebook.

The example dataset was simulated using the setup described in the paper, corresponding to SNP data for 150 diploid individuals and a chromosomal region of 30 Mb, with recombination rate from chromosome 2 and under a European demographic model (see https://www.biorxiv.org/content/10.1101/2020.04.20.029819v1 for more details).

1. import the necessary modules, including from `asmc` which is installed with the Python bindings

In [1]:
%config InlineBackend.figure_formats = ['svg']

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from asmc import (
    HMM,
    DecodingQuantities,
    DecodingParams,
    Data,
    DecodingMode,
    FastSMC,
)

2. Find the example data provided with ASMC that we will use for this notebook

In [2]:
# Get the ASMC directory and check we're in the expected place
base_dir = os.path.abspath('..')
assert 'FILES' in os.listdir(base_dir)
assert 'README.md' in os.listdir(base_dir)

file_dir = os.path.join(base_dir, 'FILES', 'FASTSMC_EXAMPLE')
name_prefix = 'out.25.n300.chr2.len30.dens1.disc10-20-2000.demoCEU.mapnorm.array'
decoding_quantities_file = os.path.join(file_dir, f'{name_prefix}.decodingQuantities.gz')
assert os.path.isfile(decoding_quantities_file)

# Ouput files will be generated alongside this notebook in a directory named 'tmp_output'
output_dir = os.path.join(base_dir, 'notebooks', 'tmp_output')
os.makedirs(output_dir, exist_ok=True)
assert(os.path.isdir(output_dir))

3. Set the required parameters

In [3]:
params = DecodingParams()
params.decodingQuantFile = decoding_quantities_file
params.inFileRoot = os.path.join(file_dir, name_prefix)
params.outFileRoot = os.path.join(output_dir, 'fastsmc_output')
params.decodingModeString = 'array'
params.decodingMode = DecodingMode.arrayFolded
params.foldData = True
params.usingCSFS = True
params.batchSize = 32
params.recallThreshold = 3
params.min_m = 1.5
params.GERMLINE = True
params.FastSMC = True
params.BIN_OUT = False
params.time = 50
params.noConditionalAgeEstimates = True
params.doPerPairMAP = True
params.doPerPairPosteriorMean = True
params.jobs = 4

4. Create the Python ASMC objects and run all four jobs.  This will probably take around 5-10 minutes on a laptop.

In [ ]:
decoding_quantities = DecodingQuantities(params.decodingQuantFile)

for job_ind in range(params.jobs):
    params.jobInd = 1 + job_ind   # params.jobInd is from 1...N
    data = Data(params, decoding_quantities, useKnownSeed=False)

    hmm = HMM(data, decoding_quantities, params, scalingSkip=1)

    fast_smc = FastSMC()
    fast_smc.run(params, data, hmm)

5. Concatenate data

In [ ]:
for job_ind in range(params.jobs):
    assert os.path.isfile(f'{params.outFileRoot}.{1 + job_ind}.{params.jobs}.FastSMC.ibd.gz')

frames = [pd.read_csv(f'{params.outFileRoot}.{x}.{params.jobs}.FastSMC.ibd.gz', sep='\t', header=None)
          for x in range(1, params.jobs + 1)]

all_data = pd.concat(frames)

col_names = ['ind1_famid', 'ind1_id', 'ind1_hap', 'ind2_famid', 'ind2_id', 'ind2_hap',
             'chromosome', 'ibd_start', 'ibd_end', 'ibd_score', 'map_est', 'post_est']

all_data.columns = col_names

# Calculate IBD segment length in number of base pairs
all_data.insert(9, 'ibd_length', all_data['ibd_end'] - all_data['ibd_start'])
all_data

6. Visualise data: here we simply bin the posterior age estimates of the IBD segments

In [ ]:
plt.xlabel("MAP age estimate (in generations)")
all_data['map_est'].hist()


In [ ]:
plt.xlabel("IBD segments length (in kbp)")
(all_data['ibd_length'] / 1000).hist(range=[1, 200])